In [ ]:
from __future__ import print_function
import codecs
import os
from os import path

import numpy as np
import tensorflow as tf

from utils import *

In [ ]:
parent_dir = path.dirname(os.getcwd())

# 1. Load Training Data, Dev Data and Test Data

In [ ]:
def load_data(data_path):
    """Loads the data stored at `data_path`.
    
    Parameters
    ----------
        data_path : str
        
    Yields
    -------
        word : list[unicode]

        score : int
    """
    with codecs.open(data_path, encoding='utf-8') as f:
        for line in f:
            line = line.lower()
            words_score = line.split("|")
            words = words_score[:-1]
            score = int(words_score[-1])
            yield words, score

In [ ]:
train_full_sentences_path = path.join(parent_dir, "data/sst/train_sentences.txt")
train_phrases_path = path.join(parent_dir, "data/sst/train_phrases.txt")
dev_full_sentences_path = path.join(parent_dir, "data/sst/dev_sentences.txt")
test_full_sentences_path = path.join(parent_dir, "data/sst/test_sentences.txt")

In [ ]:
train_full_sentences, train_full_sentences_scores = zip(*load_data(train_full_sentences_path))
train_phrases, train_phrases_scores = zip(*load_data(train_phrases_path))
train_sentences = train_full_sentences + train_phrases
train_scores = train_full_sentences_scores + train_phrases_scores

dev_full_sentences, dev_full_sentences_scores = zip(*load_data(dev_full_sentences_path))
test_full_sentences, test_full_sentences_scores = zip(*load_data(test_full_sentences_path))

Some example sentences and score from the training data
* 0 - most negative
* 2 - neutral
* 4 - most positive

In [ ]:
for sentence, score in zip(train_full_sentences[:5], train_full_sentences_scores[:5]):
    print(" ".join(sentence) + " | " + str(score))

# 2. Load embeddings

In [ ]:
embedding_path = path.join(parent_dir, 
                           "data/sst/word2vec_filtered_lower-negative300.txt")

In [ ]:
# Zero vector for padding sentence to a fixed length and for unknown words
weight_vectors = [np.zeros(300, dtype=np.float32)]
word2idx = {u"<unk>" : 0}

with codecs.open(embedding_path, encoding='utf-8') as f:
    print('loading word2vec embeddings from %s' % embedding_path)
    for line in f:
        word, vec = line.split(u' ', 1)
        word2idx[word] = len(weight_vectors)
        weight_vectors.append(np.array(vec.split(), dtype=np.float32))

# Random embedding vector for filter padding.
word2idx[u"<filter_padding>"] = len(weight_vectors)
weight_vectors.append(np.random.uniform(-0.25, 0.25, 300).astype(np.float32))

In [ ]:
we = np.asarray(weight_vectors)

# 3. Transform phrases/sentences to embedding indices

In [ ]:
train_data = pad_sequences(
    [words_to_embedding_index_with_padding(words, word2idx) for words in train_sentences],
    word2idx[u'<unk>']
)

dev_data = pad_sequences(
    [words_to_embedding_index_with_padding(words, word2idx) for words in dev_full_sentences],
    word2idx[u'<unk>']
)

test_data = pad_sequences(
    [words_to_embedding_index_with_padding(words, word2idx) for words in test_full_sentences],
    word2idx[u'<unk>']
)

# Define CNN model

In [ ]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN"""
    
    # 1. Input Data
    input_data = features["x"]
    
    # 2. Word Embedding
    channel = tf.contrib.layers.embed_sequence(
        ids=input_data,
        initializer=tf.constant_initializer(
            value=we,
            dtype=tf.float32),
        trainable=False,
        scope='embedding',
        vocab_size=we.shape[0], 
        embed_dim=we.shape[1]
    )
    
    # 3. Convolution
    branches = []
    for branch_index in range(3):
        with tf.variable_scope('CNN_Layer' + str(branch_index)):
            inference = tf.layers.conv1d(
                channel,
                filters=100,  # feature maps in the paper
                kernel_size=3 + branch_index, # filter window
                padding='VALID',
                activation=tf.nn.relu,
                kernel_initializer=tf.random_uniform_initializer(
                                    minval=-0.01,
                                    maxval=0.01,
                                    dtype=tf.float32
                                ),
                bias_initializer=tf.zeros_initializer(dtype=tf.float32)
            )
            branch = tf.reduce_max(input_tensor=inference, axis=1)
            branches.append(branch)
    network = tf.concat(values=branches, axis=1)
    
    # 5. Dropout for penultimate layer
    dropout = tf.layers.dropout(
        inputs=network, 
        rate=0.5, 
        training=mode == tf.estimator.ModeKeys.TRAIN
    )
    
    # 6. Final layer
    logits = tf.layers.dense(inputs=dropout,
                             kernel_initializer=tf.random_normal_initializer(
                                 mean=0.0, 
                                 stddev=0.01
                             ),
                             bias_initializer=tf.zeros_initializer(dtype=tf.float32),
                             units=5)
    
    # Training and Prediction
    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
  
    # Return predictions if mode is to PREDICT
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=5)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdadeltaOptimizer(
            learning_rate=1,
            rho=0.95,
            epsilon=1e-06
        )
        
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step()
        )
        
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, 
            predictions=predictions["classes"]
        )
    }
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

# Training and Evaluation

In [ ]:
train_data.shape, dev_data.shape, test_data.shape

In [ ]:
train_labels = np.asarray(train_scores, dtype=np.int32)
dev_labels = np.asarray(dev_full_sentences_scores, dtype=np.int32)
test_labels = np.asarray(test_full_sentences_scores, dtype=np.int32)

#### Create the Estimator

In [ ]:
run_config = tf.estimator.RunConfig(model_dir="../data/model/cnn/",
                                    save_checkpoints_secs=600,
                                    log_step_count_steps=1000,
                                    save_summary_steps=1000)

In [ ]:
cnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                        config=run_config)

#### Train the model

In [ ]:
num_runs = 1
best_dev_score = 0.0

In [ ]:
for run in range(num_runs):
    # train
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=50,
        num_epochs=None,
        shuffle=True
    )

    cnn_classifier.train(
        input_fn=train_input_fn,
        steps=3356
    )
    
    # Dev scores
    dev_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'x': dev_data},
        y=dev_labels,
        num_epochs=1,
        shuffle=False
    )
    dev_score = cnn_classifier.evaluate(input_fn=dev_input_fn)['accuracy']
    print('Accuracy: {0:f}'.format(dev_score))
    
    
    # Test scores
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={'x': test_data},
      y=test_labels,
      num_epochs=1,
      shuffle=False
    )
    test_score = cnn_classifier.evaluate(input_fn=test_input_fn)['accuracy']
    print('Accuracy: {0:f}'.format(test_score))
    
    if dev_score > best_dev_score:
        dev_score = dev_score
        best_model_test_score = test_score
        
    print(50 * '*')

In [ ]:
dev_score, test_score